**Полный правильный код**

In [ ]:
!pip install librosa numpy torch torchaudio tqdm textgrid openai-whisper


  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 800.5/800.5 kB 35.4 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 99.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 90.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 55.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 13.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
import librosa
import numpy as np
import whisper
import warnings
from tqdm import tqdm
from pathlib import Path
from textgrid import TextGrid, IntervalTier

# Конфигурация
DATA_DIR = "/content/audio"
SAMPLE_RATE = 16000
MIN_SILENCE_DURATION = 0.3
PAUSE_TOKEN = lambda d: f"( {round(d,2)} )"

warnings.filterwarnings("ignore", message="FP16 is not supported on CPU")
whisper_model = whisper.load_model("medium")

output_dir = Path(DATA_DIR) / "textgrids"
output_dir.mkdir(exist_ok=True)

def analyze_pauses(y, sr, words):
    frame_length = int(sr * 0.02)
    hop_length = int(sr * 0.01)
    energy = librosa.feature.rms(y=y, frame_length=frame_length, hop_length=hop_length)[0]
    times = librosa.frames_to_time(np.arange(len(energy)), sr=sr, hop_length=hop_length)
    silence_threshold = np.median(energy) * 0.5

    pauses = []
    in_silence = False
    silence_start = 0

    for t, e in zip(times, energy):
        if e < silence_threshold and not in_silence:
            in_silence = True
            silence_start = t
        elif e >= silence_threshold and in_silence:
            in_silence = False
            silence_duration = t - silence_start
            if silence_duration >= MIN_SILENCE_DURATION:
                pauses.append({
                    "start": silence_start,
                    "end": t,
                    "duration": silence_duration,
                    "source": "librosa"
                })
    return pauses

def adjust_segments(segments):
    """Корректирует сегменты, устраняя перекрытия"""
    if not segments:
        return []

    # Сначала сортируем по времени начала
    segments.sort(key=lambda x: x["start"])

    adjusted = []
    current = segments[0].copy()

    for next_seg in segments[1:]:
        if current["end"] > next_seg["start"]:
            # Если есть перекрытие, разделяем сегменты
            adjusted.append({
                "text": current["text"],
                "start": current["start"],
                "end": next_seg["start"]
            })
            current = next_seg.copy()
        else:
            adjusted.append(current)
            current = next_seg.copy()
    adjusted.append(current)

    # Удаляем сегменты с нулевой длительностью
    return [s for s in adjusted if s["end"] - s["start"] > 0.001]

def process_audio(file_path):
    file_path = Path(file_path)

    try:
        y, sr = librosa.load(file_path, sr=SAMPLE_RATE)
        duration = len(y) / sr

        result = whisper_model.transcribe(str(file_path), word_timestamps=True)
        segments = result.get("segments", [])

        words = []
        for seg in segments:
            if "words" in seg and isinstance(seg["words"], list):
                for word in seg["words"]:
                    if isinstance(word, dict) and "word" in word:
                        words.append({
                            "text": word.get("word", "").strip(),
                            "start": word.get("start", 0),
                            "end": word.get("end", 0)
                        })
            else:
                words.append({
                    "text": seg.get("text", "").strip(),
                    "start": seg.get("start", 0),
                    "end": seg.get("end", 0)
                })

        if not words:
            words.append({
                "text": "[Текст не распознан]",
                "start": 0,
                "end": duration
            })

        # Анализ пауз
        whisper_pauses = []
        for i in range(len(words)-1):
            pause_duration = words[i+1]["start"] - words[i]["end"]
            if pause_duration >= MIN_SILENCE_DURATION:
                whisper_pauses.append({
                    "start": words[i]["end"],
                    "end": words[i+1]["start"],
                    "duration": pause_duration,
                    "source": "whisper"
                })

        librosa_pauses = analyze_pauses(y, sr, words)

        # Объединение пауз
        all_pauses = whisper_pauses + librosa_pauses
        all_pauses.sort(key=lambda x: x["start"])

        unique_pauses = []
        for pause in all_pauses:
            if not unique_pauses or pause["start"] >= unique_pauses[-1]["end"]:
                unique_pauses.append(pause)
            else:
                unique_pauses[-1]["end"] = max(unique_pauses[-1]["end"], pause["end"])
                unique_pauses[-1]["duration"] = unique_pauses[-1]["end"] - unique_pauses[-1]["start"]
                unique_pauses[-1]["source"] = "combined"

        # Создание сегментов
        events = []
        for word in words:
            events.append({
                "type": "word",
                "text": word["text"],
                "start": word["start"],
                "end": word["end"]
            })

        for pause in unique_pauses:
            events.append({
                "type": "pause",
                "text": PAUSE_TOKEN(pause["duration"]),
                "start": pause["start"],
                "end": pause["end"]
            })

        events.sort(key=lambda x: x["start"])

        # Построение интервалов с проверкой перекрытий
        segments = []
        current_time = 0.0

        for event in events:
            if current_time < event["start"]:
                segments.append({
                    "text": "",
                    "start": current_time,
                    "end": event["start"]
                })

            segments.append({
                "text": event["text"],
                "start": event["start"],
                "end": event["end"]
            })
            current_time = event["end"]

        if current_time < duration:
            segments.append({
                "text": "",
                "start": current_time,
                "end": duration
            })

        # Корректировка сегментов
        segments = adjust_segments(segments)

        # Создание TextGrid
        tg = TextGrid(minTime=0, maxTime=duration)
        tier = IntervalTier(name="transcription", minTime=0, maxTime=duration)

        for s in segments:
            if s["end"] > s["start"]:
                tier.add(s["start"], s["end"], s["text"])

        tg.append(tier)

        # Сохранение
        out_path = output_dir / (file_path.stem + ".TextGrid")
        tg.write(str(out_path))
        print(f"✅ Успешно обработан: {file_path.name}")

    except Exception as e:
        print(f"❌ Ошибка при обработке {file_path.name}: {str(e)}")

if __name__ == "__main__":
    wav_files = sorted(f for f in Path(DATA_DIR).glob("*") if f.suffix.lower() == ".wav")
    print(f"🔍 Найдено {len(wav_files)} WAV-файлов для обработки")

    for wav in tqdm(wav_files):
        target_path = output_dir / (wav.stem + ".TextGrid")
        if not target_path.exists():
            process_audio(wav)
        else:
            print(f"⏩ Пропуск (уже обработан): {wav.name}")

    print("\n🎉 Обработка завершена! Результаты сохранены в:", output_dir)

100%|██████████████████████████████████████| 1.42G/1.42G [00:08<00:00, 172MiB/s]


🔍 Найдено 1 WAV-файлов для обработки


100%|██████████| 1/1 [01:03<00:00, 63.85s/it]

✅ Успешно обработан: Елисеич середина.wav

🎉 Обработка завершена! Результаты сохранены в: /content/audio/textgrids


In [ ]:
!pip install librosa
import librosa
from pathlib import Path


AUDIO_DIR = Path("/content/audio")


wav_files = sorted(AUDIO_DIR.glob("*.wav"))

total_duration = 0.0

print("Длительности отдельных файлов:\n")

for wav in wav_files:
    try:
        y, sr = librosa.load(wav, sr=None)
        duration = len(y) / sr
        total_duration += duration
        print(f"{wav.name}: {duration:.2f} сек")
    except Exception as e:
        print(f"❌ Ошибка при чтении {wav.name}: {e}")

print(f"\n📊 Общая длительность: {total_duration:.2f} сек ({total_duration/60:.2f} мин)")


Длительности отдельных файлов:

Владимир Иванович Донцов конец.wav: 277.48 сек
Владимир Иванович Донцов начало.wav: 239.96 сек
Владимир Иванович Донцов середина.wav: 240.05 сек
Елисеич конец.wav: 240.05 сек
Елисеич начало.wav: 240.19 сек
Елисеич середина.wav: 240.84 сек
Ложкарев конец.wav: 240.02 сек
Ложкарев начало.wav: 253.56 сек
Ложкарев середина.wav: 269.03 сек
Молчанов Леонид Иванович конец.wav: 240.65 сек
Молчанов Леонид Иванович начало.wav: 242.32 сек
Молчанов Леонид Иванович середина.wav: 250.13 сек
Наргулева конец.wav: 302.79 сек
Наргулева начало.wav: 240.09 сек
Наргулева середина.wav: 241.19 сек
Полуэктовна_библ_конец.wav: 240.19 сек
Полуэктовна_библ_начало.wav: 248.00 сек
Полуэктовна_библ_середина.wav: 232.62 сек
Поляничко Л начало. детство.wav: 240.31 сек
Поляничко Л середина. Байкал.wav: 243.11 сек
Поляничко Л финал. спорт.wav: 244.08 сек
Сериков Л конец.wav: 240.00 сек
Сериков Л начало.wav: 240.09 сек
Сериков Л середина.wav: 255.00 сек
Фролова конец.wav: 241.30 сек
Фролов